In [1]:
import numpy as np
import pandas as pd
import networkx as nx

In [2]:
%config InlineBackend.figure_format='retina'
%matplotlib qt 
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (8, 6)
import matplotlib.pyplot as plt

In [3]:
from CarlaBEV.envs.utils import load_planning_map, load_map

planning_map = load_planning_map()
rgbmap, _ = load_map(size=1024)

print(f"Image Size: {planning_map.shape}")
print(f"Image Size: {rgbmap.shape}")

Image Size: (1280, 1024)
Image Size: (10240, 8192, 3)


In [9]:
intersections = [
    (8642, 1564),
    (8654, 6755),
    (7250, 1552),
    (7241, 2446),
    (7242, 3652),
    (7242, 4704),
    (7257, 6773),
    (6199, 1552),
    (6197, 2439),
    (3349, 1545),
    (3350, 2456),
    (3350, 3639),
    (3335, 4714),
    (3315, 6773),
    (2456, 1563),
    (2446, 6757),
 ]

In [10]:
map_edges = {}
for i, coord in enumerate(intersections):
    x, y = coord
    map_edges[i] = (x, y)

In [6]:
from skimage.color import rgb2gray
from skimage.morphology import skeletonize

def get_midlane_coords(rgbmap):
    gray = rgb2gray(rgbmap)
    binary = gray > 0.99  # Adjust threshold if needed
    # Skeletonize to get lane centerlines
    skeleton = skeletonize(binary)
    # Get coordinates of skeleton pixels (centerline points)
    centerline_coords = np.argwhere(skeleton)
    return centerline_coords

def sample_equidistant_points(path_coords, step=100):
    """Given a list of (y, x) coords, return equidistant points along the path."""
    path_coords = np.array(path_coords)
    # Compute cumulative distance along the path
    deltas = np.diff(path_coords, axis=0)
    dists = np.sqrt((deltas**2).sum(axis=1))
    cumdist = np.insert(np.cumsum(dists), 0, 0)
    total_dist = cumdist[-1]
    n_points = int(total_dist // step)
    if n_points < 2:
        return path_coords  # Path too short, return as is
    sample_dists = np.linspace(0, total_dist, n_points)
    sampled_points = np.empty((n_points, 2))
    sampled_points[0] = path_coords[0]
    sampled_points[-1] = path_coords[-1]
    j = 1
    for i in range(1, n_points-1):
        d = sample_dists[i]
        while cumdist[j] < d:
            j += 1
        ratio = (d - cumdist[j-1]) / (cumdist[j] - cumdist[j-1])
        sampled_points[i] = path_coords[j-1] + ratio * (path_coords[j] - path_coords[j-1])
    return sampled_points.astype(int)

In [7]:
centerline_coords = get_midlane_coords(rgbmap)

In [17]:
plt.imshow(rgbmap)
plt.scatter(centerline_coords[:,1], centerline_coords[:,0], c='gray', s=0.1, label='All Centerlines')
for idx, p in enumerate(intersections):
    plt.scatter(p[1], p[0], c='red', s=20)
    plt.annotate(str(idx), (p[1], p[0]), color='k', fontsize=10, weight='bold')
plt.legend()
plt.title("Closest Centerlines & Midpoints Between Intersections")
plt.show()

In [16]:
# 1. Find the closest centerline point for each intersection
intersection_nodes = []
for intersection in intersections:
    dists = np.linalg.norm(centerline_coords - intersection, axis=1)
    idx_min = np.argmin(dists)
    intersection_nodes.append(tuple(centerline_coords[idx_min]))

# 2. Build a pixel adjacency graph for centerline_coords (for pathfinding only)
pixel_graph = nx.Graph()
centerline_set = set(map(tuple, centerline_coords))
for y, x in centerline_coords:
    for dy in [-1, 0, 1]:
        for dx in [-1, 0, 1]:
            if dy == 0 and dx == 0:
                continue
            neighbor = (y + dy, x + dx)
            if neighbor in centerline_set:
                pixel_graph.add_edge((y, x), neighbor)

# 3. Precompute a set of "blocked" pixels for each intersection (within a radius)
block_radius = 100  # pixels
blocked_pixels = []
for node in intersection_nodes:
    y0, x0 = node
    pixels = set()
    for dy in range(-block_radius, block_radius+1):
        for dx in range(-block_radius, block_radius+1):
            if dy**2 + dx**2 <= block_radius**2:
                pixels.add((y0+dy, x0+dx))
    blocked_pixels.append(pixels)

# 4. Build the intersection graph
G_intersections = nx.Graph()
for idx in range(len(intersections)):
    G_intersections.add_node(idx, pos=intersections[idx])

for i, node_a in enumerate(intersection_nodes):
    for j, node_b in enumerate(intersection_nodes):
        if i < j:
            try:
                path = nx.shortest_path(pixel_graph, node_a, node_b)
                path_set = set(path[1:-1])
                blocked = False
                for k, block in enumerate(blocked_pixels):
                    if k != i and k != j and path_set & block:
                        blocked = True
                        break
                if not blocked:
                    G_intersections.add_edge(i, j)
            except nx.NetworkXNoPath:
                continue

print("Direct intersection pairs (by index):", list(G_intersections.edges))

# Visualization
plt.imshow(rgbmap)
for idx, p in enumerate(intersections):
    plt.scatter(p[1], p[0], c='red', s=20)
    plt.annotate(str(idx), (p[1], p[0]), color='k', fontsize=10, weight='bold')
for i, j in G_intersections.edges:
    a = intersections[i]
    b = intersections[j]
    plt.plot([a[1], b[1]], [a[0], b[0]], 'c-', lw=2)
plt.title("Intersection Graph (nodes=intersections, edges=direct paths)")
plt.show()

Direct intersection pairs (by index): [(0, 1), (0, 2), (1, 6), (2, 3), (2, 7), (3, 4), (3, 8), (4, 5), (4, 11), (5, 6), (5, 12), (6, 13), (7, 8), (7, 9), (8, 10), (9, 10), (9, 14), (10, 11), (11, 12), (12, 13), (13, 15), (14, 15)]


In [29]:
options = {
    "font_size": 18,
    "node_size": 1000,
    "node_color": "white",
    "edgecolors": "black",
    "linewidths": 5,
    "width": 5,
}
nx.draw_networkx(G_intersections, map_edges, **options)

# Set margins for the axes so that nodes aren't clipped
ax = plt.gca()
ax.margins(0.20)
plt.axis("off")
plt.show()

In [15]:
# ...existing code...

start_idx = 0  # index of start intersection
end_idx = 5    # index of goal intersection

# 1. Find the shortest path (by number of edges)
route = nx.shortest_path(G_intersections, source=start_idx, target=end_idx)
print("Route (intersection indices):", route)

# 2. For each consecutive pair, find pixel path and discretize
full_waypoints = []
for i in range(len(route) - 1):
    idx_a = route[i]
    idx_b = route[i+1]
    node_a = intersection_nodes[idx_a]
    node_b = intersection_nodes[idx_b]
    pixel_path = nx.shortest_path(pixel_graph, node_a, node_b)
    waypoints = sample_equidistant_points(pixel_path, step=100)  # step in pixels
    if i > 0:
        # Avoid duplicating the first point of each segment except the first
        waypoints = waypoints[1:]
    full_waypoints.extend(waypoints)

full_waypoints = np.array(full_waypoints)
print("Total waypoints:", len(full_waypoints))

# 3. Visualize the full route
plt.imshow(rgbmap)
for idx, p in enumerate(intersections):
    plt.scatter(p[1], p[0], c='red', s=80, zorder=3)
    plt.annotate(str(idx), (p[1], p[0]), color='k', fontsize=12, weight='bold', zorder=4)
plt.plot(full_waypoints[:,1], full_waypoints[:,0], 'orange', lw=2, zorder=5, label='Discretized Route')
plt.scatter(full_waypoints[:,1], full_waypoints[:,0], c='b', s=1, zorder=6, label='Waypoints')
plt.title(f"Discretized Route from {start_idx} to {end_idx}")
plt.axis('off')
plt.legend()
plt.show()

Route (intersection indices): [0, 1, 6, 5]
Total waypoints: 84


In [14]:
def compute_lane_offsets(waypoints, lane_width=6.0):
    """
    Given centerline waypoints (N,2), return left and right lane waypoints.
    lane_width: total width of both lanes (in pixels, adjust as needed)
    """
    waypoints = np.array(waypoints)
    left_lane = []
    right_lane = []
    offset = lane_width / 2.0

    for i in range(len(waypoints)):
        # Compute direction vector
        if i == 0:
            direction = waypoints[i+1] - waypoints[i]
        elif i == len(waypoints) - 1:
            direction = waypoints[i] - waypoints[i-1]
        else:
            direction = waypoints[i+1] - waypoints[i-1]
        direction = direction / np.linalg.norm(direction)
        # Normal vector (perpendicular, 2D)
        normal = np.array([-direction[1], direction[0]])
        # Offset points
        left_lane.append(waypoints[i] + offset * normal)
        right_lane.append(waypoints[i] - offset * normal)
    return np.array(left_lane), np.array(right_lane)

# Usage
lane_width_pixels = 75  # Adjust to your map's scale
left_lane, right_lane = compute_lane_offsets(full_waypoints, lane_width=lane_width_pixels)

# Visualization
plt.imshow(rgbmap)
plt.plot(full_waypoints[:,1], full_waypoints[:,0], 'orange', lw=2, label='Centerline')
plt.plot(left_lane[:,1], left_lane[:,0], 'g--', lw=2, label='Left Lane')
plt.plot(right_lane[:,1], right_lane[:,0], 'b--', lw=2, label='Right Lane')
plt.scatter(full_waypoints[:,1], full_waypoints[:,0], c='orange', s=1)
plt.scatter(left_lane[:,1], left_lane[:,0], c='g', s=1)
plt.scatter(right_lane[:,1], right_lane[:,0], c='b', s=1)
plt.legend()
plt.axis('off')
plt.title("Centerline and Split Lane Routes")
plt.show()